In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from brian2 import *
from network_code.utils import *
from network_code.parameters import *
from network_code.network import *
from network_code.simulations import *

matplotlib.rcParams['axes.spines.top'] = False
matplotlib.rcParams['axes.spines.right'] = False
plt.rc('font', size=20)
plt.rc('axes', titlesize=20, labelsize=20)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)

In [ ]:
net_params = get_default_net_params()
test_params = get_dft_test_params()
test_params['sim_time'].set_param(20)
test_params['record_spikes'].set_param(True)

built_network, all_used_params = build_network(net_params)
built_network, test_params = record_network(built_network, all_used_params, test_params)
tested_network, test_params = run_simulation(built_network, test_params)
stats, time_traces, test_params = analyze_simulation(tested_network, test_params)

In [ ]:
start_time = stats['start_time']
start_stats = stats['start_stats']
end_time = stats['end_time']
sim_dt = test_params['sim_dt'].get_param()
the_time = time_traces['the_time']
timeline = the_time[int(start_time/sim_dt):int(end_time/sim_dt)+1]

# Save relevant time traces from sample run
for key in list(time_traces.keys()):
    globals()[key] = time_traces[key]
    np.savetxt(f'samples/network_traces/{key}.csv', time_traces[key], delimiter=',')

# Select a SPW
spw_center = stats['event_peak_index'][11]*sim_dt/second
# Cut short window
plot_range_times = [spw_center-0.4,spw_center+0.2]
plot_range_idx = np.arange(round((spw_center-0.4)/sim_dt *second),round((spw_center+0.2)/sim_dt *second),1)
# Cut long window
plot_range_long_times = [spw_center-3,spw_center+3]
plot_range_long_idx = np.arange(round((spw_center-3)/sim_dt *second),round((spw_center+3)/sim_dt *second),1)
# Cut pre-SPW baseline
baseline_idx = np.arange(round((spw_center-0.5)/sim_dt *second),round((spw_center-0.3)/sim_dt *second),1)

# Calculate pre-SPW adaptation values
baseline_ada_a = np.mean(stm_a_adp[baseline_idx])
baseline_ada_t = np.mean(stm_t_adp[baseline_idx])
baseline_ada_b = np.mean(stm_b_adp[baseline_idx])
baseline_ada_c = np.mean(stm_c_adp[baseline_idx])

In [ ]:
fig=plt.figure(figsize=(9,18))

ax1=fig.add_subplot(16,1,(1,6))
ax1.plot(timeline[plot_range_idx],rtm_a[plot_range_idx],color='goldenrod')
ax1.plot(timeline[plot_range_idx],rtm_t[plot_range_idx],color='magenta')
ax1.plot(timeline[plot_range_idx],rtm_b[plot_range_idx],color='blue')
ax1.plot(timeline[plot_range_idx],rtm_c[plot_range_idx],color='green')
ax1.spines['bottom'].set_visible(False)
ax1.set_xlim([spw_center-0.4,spw_center+0.2])
ax1.set_ylim([-1,75])
ax1.set_xticks([])
ax1.set_yticks([0,20,40,60])
ax1.set_ylabel('Firing rate [spikes/s]',fontsize=24)
ax1.set_title('Network Activity',fontsize=24)
ax12 = ax1.inset_axes([0.07, 0.5, 0.5, 0.4])
ax12.set_ylabel('Low-pass LFP [pA]',fontsize=18)
ax12.plot(timeline[plot_range_long_idx],lowpass_lfp[plot_range_long_idx],color='black')
ax12.hlines(y=180, xmin=plot_range_times[0], xmax=plot_range_times[1], color='gray', linestyle='--')
ax12.hlines(y=-10, xmin=plot_range_times[0], xmax=plot_range_times[1], color='gray', linestyle='--')
ax12.vlines(x=plot_range_times[0], ymin=-10, ymax=180, color='gray', linestyle='--')
ax12.vlines(x=plot_range_times[1], ymin=-10, ymax=180, color='gray', linestyle='--')
ax12.tick_params(axis='both', which='major', labelsize=12)
ax12.spines['top'].set_visible(True)
ax12.spines['left'].set_visible(True)
ax12.spines['right'].set_visible(True)
ax12.spines['bottom'].set_visible(True)
ax12.spines['top'].set_linestyle('-')
ax12.spines['right'].set_linestyle('-')
ax12.spines['bottom'].set_linestyle('-')
ax12.spines['left'].set_linestyle('-')
ax12.set_xticks([])
ax12.set_yticks([])
ax12.hlines(y=176.5, xmin=plot_range_times[0]+1, xmax=plot_range_times[0]+2, color='k', linestyle='-')
ax12.text(plot_range_times[0]+1.5, 190, '1 s', backgroundcolor='none', fontsize=18, va='center', ha='center')
ax1.hlines(y=60, xmin=plot_range_times[0]+0.52, xmax=plot_range_times[0]+0.57, color='k', linestyle='-')
ax1.text(plot_range_times[0]+0.5, 64.5, '50 ms', backgroundcolor='none', fontsize=24, va='top', ha='left')

ax4=fig.add_subplot(16,1,(7,8))
ax4.plot(spm_a[0][(spm_a[1]>-1) & (spm_a[1]<51)], spm_a[1][(spm_a[1]>-1) & (spm_a[1]<51)],'.',color='goldenrod')
ax4.set_xlim(plot_range_times)
ax4.set_ylim((-1,51))
ax4.spines['bottom'].set_visible(False)
ax4.set_xticks([])
ax4.spines['left'].set_visible(False)
ax4.set_yticks([])

ax5=fig.add_subplot(16,1,(9,10))
ax5.plot(spm_t[0][(spm_t[1]>49) & (spm_t[1]<101)], spm_t[1][(spm_t[1]>49) & (spm_t[1]<101)],'.',color='magenta')
ax5.set_xlim(plot_range_times)
ax5.set_ylim((49,101))
ax5.spines['bottom'].set_visible(False)
ax5.set_xticks([])
ax5.spines['left'].set_visible(False)
ax5.set_yticks([])

ax6=fig.add_subplot(16,1,11)
ax6.plot(spm_b[0][(spm_b[1]>-1) & (spm_b[1]<26)], spm_b[1][(spm_b[1]>-1) & (spm_b[1]<26)],'.',color='blue')
ax6.set_xlim(plot_range_times)
ax6.set_ylim((-1,26))
ax6.spines['bottom'].set_visible(False)
ax6.set_xticks([])
ax6.spines['left'].set_visible(False)
ax6.set_yticks([])

ax7=fig.add_subplot(16,1,12)
ax7.plot(spm_c[0][(spm_c[1]>-1) & (spm_c[1]<26)], spm_c[1][(spm_c[1]>-1) & (spm_c[1]<26)],'.',color='green')
ax7.set_xlim(plot_range_times)
ax7.set_ylim((-1,26))
ax7.spines['bottom'].set_visible(False)
ax7.set_xticks([])
ax7.spines['left'].set_visible(False)
ax7.set_yticks([])

ax3=fig.add_subplot(16,1,(13,16))
ax3.plot(timeline[plot_range_idx],stm_a_adp[plot_range_idx]/baseline_ada_a,color='goldenrod')
ax3.plot(timeline[plot_range_idx],stm_t_adp[plot_range_idx]/baseline_ada_t,color='magenta')
ax3.plot(timeline[plot_range_idx],stm_b_adp[plot_range_idx]/baseline_ada_b,color='blue')
ax3.plot(timeline[plot_range_idx],stm_c_adp[plot_range_idx]/baseline_ada_c,color='green')
ax3.spines['bottom'].set_visible(False)
ax3.set_xticks([])
ax3.set_ylabel('Adaptation Increment',fontsize=24)

plt.tight_layout()
plt.show()

In [ ]:
net_params = get_default_net_params()

# Adjustment of default parameters for synaptic depression
net_params['J_spi_a'].set_param(120)
net_params['J_spi_t'].set_param(180)
net_params['g_ab'].set_param(2)
net_params['g_tb'].set_param(0.8)
net_params['g_bt'].set_param(0.55)
net_params['g_ba'].set_param(0.7)
net_params['g_aa'].set_param(0.265)
net_params['g_at'].set_param(0.265)
net_params['g_ta'].set_param(0.265)
net_params['g_tt'].set_param(0.265)

test_params = get_dft_test_params()
test_params['sim_time'].set_param(20)
test_params['record_spikes'].set_param(True)

built_network, all_used_params = build_network(net_params,plastic_ee=True)
built_network, test_params = record_network(built_network, all_used_params, test_params)
tested_network, test_params = run_simulation(built_network, test_params)
stats, time_traces, test_params = analyze_simulation(tested_network, test_params)

In [ ]:
start_time = stats['start_time']
start_stats = stats['start_stats']
end_time = stats['end_time']
sim_dt = test_params['sim_dt'].get_param()
the_time = time_traces['the_time']
timeline = the_time[int(start_time/sim_dt):int(end_time/sim_dt)+1]

# Save relevant time traces from sample run
for key in list(time_traces.keys()):
    globals()[key] = time_traces[key]
    np.savetxt(f'samples/network_traces/{key}_dep.csv', time_traces[key], delimiter=',')

# Select a SPW
spw_center = stats['event_peak_index'][7]*sim_dt/second
# Cut short window
plot_range_times = [spw_center-0.4,spw_center+0.2]
plot_range_idx = np.arange(round((spw_center-0.4)/sim_dt *second),round((spw_center+0.2)/sim_dt *second),1)
# Cut long window
plot_range_long_times = [spw_center-3,spw_center+3]
plot_range_long_idx = np.arange(round((spw_center-3)/sim_dt *second),round((spw_center+3)/sim_dt *second),1)
# Cut pre-SPW baseline
baseline_idx = np.arange(round((spw_center-0.5)/sim_dt *second),round((spw_center-0.3)/sim_dt *second),1)

# Calculate pre-SPW adaptation values
baseline_ada_a = np.mean(stm_a_adp[baseline_idx])
baseline_ada_t = np.mean(stm_t_adp[baseline_idx])
baseline_ada_b = np.mean(stm_b_adp[baseline_idx])
baseline_ada_c = np.mean(stm_c_adp[baseline_idx])

In [ ]:
fig=plt.figure(figsize=(9,18))

ax1=fig.add_subplot(16,1,(1,6))
ax1.plot(timeline[plot_range_idx],rtm_a[plot_range_idx],color='goldenrod')
ax1.plot(timeline[plot_range_idx],rtm_t[plot_range_idx],color='magenta')
ax1.plot(timeline[plot_range_idx],rtm_b[plot_range_idx],color='blue')
ax1.plot(timeline[plot_range_idx],rtm_c[plot_range_idx],color='green')
ax1.spines['bottom'].set_visible(False)
ax1.set_xlim([spw_center-0.4,spw_center+0.2])
ax1.set_ylim([-1,75])
ax1.set_xticks([])
ax1.set_yticks([0,20,40,60])
ax1.set_ylabel('Firing rate [spikes/s]',fontsize=24)
ax1.set_title('Network Activity',fontsize=24)
ax12 = ax1.inset_axes([0.07, 0.5, 0.5, 0.4])
ax12.set_ylabel('Low-pass LFP [pA]',fontsize=18)
ax12.plot(timeline[plot_range_long_idx],lowpass_lfp[plot_range_long_idx],color='black')
ax12.hlines(y=180, xmin=plot_range_times[0], xmax=plot_range_times[1], color='gray', linestyle='--')
ax12.hlines(y=-10, xmin=plot_range_times[0], xmax=plot_range_times[1], color='gray', linestyle='--')
ax12.vlines(x=plot_range_times[0], ymin=-10, ymax=180, color='gray', linestyle='--')
ax12.vlines(x=plot_range_times[1], ymin=-10, ymax=180, color='gray', linestyle='--')
ax12.tick_params(axis='both', which='major', labelsize=12)
ax12.spines['top'].set_visible(True)
ax12.spines['left'].set_visible(True)
ax12.spines['right'].set_visible(True)
ax12.spines['bottom'].set_visible(True)
ax12.spines['top'].set_linestyle('-')
ax12.spines['right'].set_linestyle('-')
ax12.spines['bottom'].set_linestyle('-')
ax12.spines['left'].set_linestyle('-')
ax12.set_xticks([])
ax12.set_yticks([])
ax12.hlines(y=161.5, xmin=plot_range_times[0]+1, xmax=plot_range_times[0]+2, color='k', linestyle='-')
ax12.text(plot_range_times[0]+1.5, 175, '1 s', backgroundcolor='none', fontsize=18, va='center', ha='center')
ax1.hlines(y=60, xmin=plot_range_times[0]+0.52, xmax=plot_range_times[0]+0.57, color='k', linestyle='-')
ax1.text(plot_range_times[0]+0.5, 64.5, '50 ms', backgroundcolor='none', fontsize=24, va='top', ha='left')

ax4=fig.add_subplot(16,1,(7,8))
ax4.plot(spm_a[0][(spm_a[1]>-1) & (spm_a[1]<51)], spm_a[1][(spm_a[1]>-1) & (spm_a[1]<51)],'.',color='goldenrod')
ax4.set_xlim(plot_range_times)
ax4.set_ylim((-1,51))
ax4.spines['bottom'].set_visible(False)
ax4.set_xticks([])
ax4.spines['left'].set_visible(False)
ax4.set_yticks([])

ax5=fig.add_subplot(16,1,(9,10))
ax5.plot(spm_t[0][(spm_t[1]>49) & (spm_t[1]<101)], spm_t[1][(spm_t[1]>49) & (spm_t[1]<101)],'.',color='magenta')
ax5.set_xlim(plot_range_times)
ax5.set_ylim((49,101))
ax5.spines['bottom'].set_visible(False)
ax5.set_xticks([])
ax5.spines['left'].set_visible(False)
ax5.set_yticks([])

ax6=fig.add_subplot(16,1,11)
ax6.plot(spm_b[0][(spm_b[1]>-1) & (spm_b[1]<26)], spm_b[1][(spm_b[1]>-1) & (spm_b[1]<26)],'.',color='blue')
ax6.set_xlim(plot_range_times)
ax6.set_ylim((-1,26))
ax6.spines['bottom'].set_visible(False)
ax6.set_xticks([])
ax6.spines['left'].set_visible(False)
ax6.set_yticks([])

ax7=fig.add_subplot(16,1,12)
ax7.plot(spm_c[0][(spm_c[1]>-1) & (spm_c[1]<26)], spm_c[1][(spm_c[1]>-1) & (spm_c[1]<26)],'.',color='green')
ax7.set_xlim(plot_range_times)
ax7.set_ylim((-1,26))
ax7.spines['bottom'].set_visible(False)
ax7.set_xticks([])
ax7.spines['left'].set_visible(False)
ax7.set_yticks([])

ax3=fig.add_subplot(16,1,(13,16))
ax3.plot(timeline[plot_range_idx],stm_a_adp[plot_range_idx]/baseline_ada_a,color='goldenrod')
ax3.plot(timeline[plot_range_idx],stm_t_adp[plot_range_idx]/baseline_ada_t,color='magenta')
ax3.plot(timeline[plot_range_idx],stm_b_adp[plot_range_idx]/baseline_ada_b,color='blue')
ax3.plot(timeline[plot_range_idx],stm_c_adp[plot_range_idx]/baseline_ada_c,color='green')
ax3.spines['bottom'].set_visible(False)
ax3.set_xticks([])
ax3.set_ylabel('Adaptation Increment',fontsize=24)

plt.tight_layout()
plt.show()

In [ ]:
net_params = get_default_net_params()

# Adjustment of default parameters for heterogeneity
net_params['g_ba'].set_param(0.55)
net_params['g_bt'].set_param(0.75)
net_params['g_tb'].set_param(0.7)
net_params['g_ab'].set_param(2.1)
net_params['g_cc'].set_param(2)
net_params['g_cb'].set_param(4.5)
net_params['curr_bg_t'].set_param(295)

test_params = get_dft_test_params()
test_params['sim_time'].set_param(20)
test_params['record_spikes'].set_param(True)

built_network, all_used_params = build_network(net_params,heterogenous=True)
built_network, test_params = record_network(built_network, all_used_params, test_params)
tested_network, test_params = run_simulation(built_network, test_params)
stats, time_traces, test_params = analyze_simulation(tested_network, test_params)

In [ ]:
start_time = stats['start_time']
start_stats = stats['start_stats']
end_time = stats['end_time']
sim_dt = test_params['sim_dt'].get_param()
the_time = time_traces['the_time']
timeline = the_time[int(start_time/sim_dt):int(end_time/sim_dt)+1]

# Save relevant time traces from sample run
for key in list(time_traces.keys()):
    globals()[key] = time_traces[key]
    np.savetxt(f'samples/network_traces/{key}_het.csv', time_traces[key], delimiter=',')

# Select a SPW
spw_center = stats['event_peak_index'][9]*sim_dt/second
# Cut short window
plot_range_times = [spw_center-0.4,spw_center+0.2]
plot_range_idx = np.arange(round((spw_center-0.4)/sim_dt *second),round((spw_center+0.2)/sim_dt *second),1)
# Cut long window
plot_range_long_times = [spw_center-3,spw_center+3]
plot_range_long_idx = np.arange(round((spw_center-3)/sim_dt *second),round((spw_center+3)/sim_dt *second),1)
# Cut pre-SPW baseline
baseline_idx = np.arange(round((spw_center-0.5)/sim_dt *second),round((spw_center-0.3)/sim_dt *second),1)

# Calculate pre-SPW adaptation values
baseline_ada_a = np.mean(stm_a_adp[baseline_idx])
baseline_ada_t = np.mean(stm_t_adp[baseline_idx])
baseline_ada_b = np.mean(stm_b_adp[baseline_idx])
baseline_ada_c = np.mean(stm_c_adp[baseline_idx])

In [ ]:
fig=plt.figure(figsize=(9,18))

ax1=fig.add_subplot(16,1,(1,6))
ax1.plot(timeline[plot_range_idx],rtm_a[plot_range_idx],color='goldenrod')
ax1.plot(timeline[plot_range_idx],rtm_t[plot_range_idx],color='magenta')
ax1.plot(timeline[plot_range_idx],rtm_b[plot_range_idx],color='blue')
ax1.plot(timeline[plot_range_idx],rtm_c[plot_range_idx],color='green')
ax1.spines['bottom'].set_visible(False)
ax1.set_xlim([spw_center-0.4,spw_center+0.2])
ax1.set_ylim([-1,95])
ax1.set_xticks([])
ax1.set_yticks([0,20,40,60,80])
ax1.set_ylabel('Firing rate [spikes/s]',fontsize=24)
ax1.set_title('Network Activity',fontsize=24)
ax12 = ax1.inset_axes([0.07, 0.5, 0.5, 0.4])
ax12.set_ylabel('Low-pass LFP [pA]',fontsize=18)
ax12.plot(timeline[plot_range_long_idx],lowpass_lfp[plot_range_long_idx],color='black')
ax12.hlines(y=220, xmin=plot_range_times[0], xmax=plot_range_times[1], color='gray', linestyle='--')
ax12.hlines(y=-10, xmin=plot_range_times[0], xmax=plot_range_times[1], color='gray', linestyle='--')
ax12.vlines(x=plot_range_times[0], ymin=-10, ymax=220, color='gray', linestyle='--')
ax12.vlines(x=plot_range_times[1], ymin=-10, ymax=220, color='gray', linestyle='--')
ax12.tick_params(axis='both', which='major', labelsize=12)
ax12.spines['top'].set_visible(True)
ax12.spines['left'].set_visible(True)
ax12.spines['right'].set_visible(True)
ax12.spines['bottom'].set_visible(True)
ax12.spines['top'].set_linestyle('-')
ax12.spines['right'].set_linestyle('-')
ax12.spines['bottom'].set_linestyle('-')
ax12.spines['left'].set_linestyle('-')
ax12.set_xticks([])
ax12.set_yticks([])
ax12.hlines(y=216.5, xmin=plot_range_times[0]+1, xmax=plot_range_times[0]+2, color='k', linestyle='-')
ax12.text(plot_range_times[0]+1.5, 230, '1 s', backgroundcolor='none', fontsize=18, va='center', ha='center')
ax1.hlines(y=80, xmin=plot_range_times[0]+0.52, xmax=plot_range_times[0]+0.57, color='k', linestyle='-')
ax1.text(plot_range_times[0]+0.5, 86.5, '50 ms', backgroundcolor='none', fontsize=24, va='top', ha='left')

ax4=fig.add_subplot(16,1,(7,8))
ax4.plot(spm_a[0][(spm_a[1]>-1) & (spm_a[1]<51)], spm_a[1][(spm_a[1]>-1) & (spm_a[1]<51)],'.',color='goldenrod')
ax4.set_xlim(plot_range_times)
ax4.set_ylim((-1,51))
ax4.spines['bottom'].set_visible(False)
ax4.set_xticks([])
ax4.spines['left'].set_visible(False)
ax4.set_yticks([])

ax5=fig.add_subplot(16,1,(9,10))
ax5.plot(spm_t[0][(spm_t[1]>49) & (spm_t[1]<101)], spm_t[1][(spm_t[1]>49) & (spm_t[1]<101)],'.',color='magenta')
ax5.set_xlim(plot_range_times)
ax5.set_ylim((49,101))
ax5.spines['bottom'].set_visible(False)
ax5.set_xticks([])
ax5.spines['left'].set_visible(False)
ax5.set_yticks([])

ax6=fig.add_subplot(16,1,11)
ax6.plot(spm_b[0][(spm_b[1]>-1) & (spm_b[1]<26)], spm_b[1][(spm_b[1]>-1) & (spm_b[1]<26)],'.',color='blue')
ax6.set_xlim(plot_range_times)
ax6.set_ylim((-1,26))
ax6.spines['bottom'].set_visible(False)
ax6.set_xticks([])
ax6.spines['left'].set_visible(False)
ax6.set_yticks([])

ax7=fig.add_subplot(16,1,12)
ax7.plot(spm_c[0][(spm_c[1]>-1) & (spm_c[1]<26)], spm_c[1][(spm_c[1]>-1) & (spm_c[1]<26)],'.',color='green')
ax7.set_xlim(plot_range_times)
ax7.set_ylim((-1,26))
ax7.spines['bottom'].set_visible(False)
ax7.set_xticks([])
ax7.spines['left'].set_visible(False)
ax7.set_yticks([])

ax3=fig.add_subplot(16,1,(13,16))
ax3.plot(timeline[plot_range_idx],stm_a_adp[plot_range_idx]/baseline_ada_a,color='goldenrod')
ax3.plot(timeline[plot_range_idx],stm_t_adp[plot_range_idx]/baseline_ada_t,color='magenta')
ax3.plot(timeline[plot_range_idx],stm_b_adp[plot_range_idx]/baseline_ada_b,color='blue')
ax3.plot(timeline[plot_range_idx],stm_c_adp[plot_range_idx]/baseline_ada_c,color='green')
ax3.spines['bottom'].set_visible(False)
ax3.set_xticks([])
ax3.set_ylabel('Adaptation Increment',fontsize=24)

plt.tight_layout()
plt.show()